In [1]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
%matplotlib notebook

In [2]:
# import the covid date csv & clean
covid_df= pd.read_csv('United_States_COVID-19_Cases_and_Deaths_by_State_over_Time.csv',thousands=',', parse_dates=[0])
covid_df.drop(['consent_cases', 'consent_deaths', 'created_at', 'conf_cases','prob_cases', 'conf_death','pnew_case','prob_death', 'pnew_death'], axis= 1, inplace = True)
covid_df.rename(columns={'submission_date':'Submission Date','state':'State','tot_cases':'Total Cases','new_case':'Daily New Cases','tot_death':'Total Deaths','new_death': 'Daily New Deaths'}, inplace = True)

covid_df.sort_values(['Submission Date','State'])

,Submission Date,State,Total Cases,Daily New Cases,Total Deaths,Daily New Deaths
3599,2020-01-22,AK,0,0,0,0
9197,2020-01-22,AL,33,33,0,0
3491,2020-01-22,AR,0,0,0,0
4833,2020-01-22,AS,0,0,0,0
27708,2020-01-22,AZ,0,0,0,0
...,...,...,...,...,...,...
13751,2021-08-27,VT,25751,0,264,0
23229,2021-08-27,WA,550988,4267,6507,36
19432,2021-08-27,WI,727353,2636,8438,9
4319,2021-08-27,WV,184682,1328,3054,5


In [4]:
# create groupby date to aggregate data from all state to reflect federal daily data
# reset index so date can be used for plotting
covid_df_grouped_date = covid_df.groupby(['Submission Date']).sum().reset_index()
covid_df_grouped_date.to_csv('OutputData/COVID.csv', index=False)
covid_df_grouped_date

,Submission Date,Total Cases,Daily New Cases,Total Deaths,Daily New Deaths
0,2020-01-22,33,33,0,0
1,2020-01-23,35,2,0,0
2,2020-01-24,36,1,0,0
3,2020-01-25,39,3,0,0
4,2020-01-26,42,3,0,0
...,...,...,...,...,...
579,2021-08-23,38037577,112020,629220,703
580,2021-08-24,38192607,154363,630408,1184
581,2021-08-25,38360956,162485,631563,1138
582,2021-08-26,38531770,170229,632786,1192


In [5]:
# covid data numerical description pane
covid_df_grouped_date.describe()

,Total Cases,Daily New Cases,Total Deaths,Daily New Deaths
count,5.840000e+02,584.000000,584.000000,584.000000
mean,1.581296e+07,66116.758562,311779.243151,1073.820205
std,1.369797e+07,63095.759432,224058.585529,926.252099
min,3.300000e+01,1.000000,0.000000,0.000000
25%,2.216883e+06,24240.000000,120195.000000,481.250000
50%,1.035425e+07,46418.500000,247403.000000,789.000000
75%,3.077049e+07,76227.750000,563051.250000,1363.500000
max,3.870930e+07,292713.000000,634157.000000,4169.000000


In [6]:
# find max and min date for covid data to use as parameters for stock history
mindate = covid_df_grouped_date['Submission Date'].min()
maxdate = covid_df_grouped_date['Submission Date'].max()

In [7]:
# generate list of sp500 indices
tickerlist = ['SPY', 'XLF', 'XLU', 'XLK', 'XLE', 'XLI',
             'XLB', 'XLP', 'XLV', 'XLY', 'XLRE', 'XLC']

In [8]:
# generate list of tickers
tickers = [yf.Ticker(tick) for tick in tickerlist]

In [9]:
# store full name of each index in list
namelist = []
for tick in tickers:
    try:
        name = tick.info['longName']
        namelist.append(name)
    except KeyError:
        namelist.append(None)
        print(f"there's no name listed for {tick}")

In [10]:
# create dataframe to correspond tickers and full names
name_df = pd.DataFrame({'Ticker': tickerlist,
                       'Full Name': namelist})
name_df

,Ticker,Full Name
0,SPY,SPDR S&P 500 ETF Trust
1,XLF,Financial Select Sector SPDR Fund
2,XLU,Utilities Select Sector SPDR Fund
3,XLK,Technology Select Sector SPDR Fund
4,XLE,Energy Select Sector SPDR Fund
5,XLI,Industrial Select Sector SPDR Fund
6,XLB,Materials Select Sector SPDR Fund
7,XLP,Consumer Staples Select Sector SPDR Fund
8,XLV,Health Care Select Sector SPDR Fund
9,XLY,Consumer Discretionary Select Sector SPDR Fund


In [11]:
# pull price history for each index
histlist = [tick.history(start=mindate, end=maxdate) for tick in tickers]

In [12]:
# reset index for each dataframe and set date format
for df in histlist:
    df.reset_index(inplace=True)
    df['Date'] = pd.to_datetime(df['Date'])
    df.rename(columns={"Date":"Submission Date"},inplace = True)

In [13]:
# add columns to history dataframes
counter = 0

for df in histlist:
    df['Amplitude'] = df['High'] - df['Low']
    df['Daily Change'] = df['Close'] - df['Open']
    df['Daily Change %'] = round(((df['Daily Change']/df['Open']) * 100), 2)
    df['Ticker'] = tickerlist[counter]
    df.drop(['Dividends', 'Stock Splits'], axis=1, inplace=True)
    
    counter +=1

In [14]:
combo_list = []
for df in histlist:
    combo = pd.merge(df, covid_df_grouped_date, on='Submission Date', how='inner')
    combo_list.append(combo)
    combo.to_csv('OutputData/'+ combo['Ticker'][0] +'COVID.csv', index=False)

In [15]:
print(tickerlist[0])
combo_list[5]


SPY


,Submission Date,Open,High,Low,Close,Volume,Amplitude,Daily Change,Daily Change %,Ticker,Total Cases,Daily New Cases,Total Deaths,Daily New Deaths
0,2020-01-22,81.528838,81.713889,80.817863,80.876297,7490500,0.896027,-0.652541,-0.80,XLI,33,33,0,0
1,2020-01-23,80.769168,81.859988,80.476982,81.752853,10758500,1.383006,0.983685,1.22,XLI,35,2,0,0
2,2020-01-24,81.908681,81.976857,80.769168,81.373016,13402200,1.207688,-0.535665,-0.65,XLI,36,1,0,0
3,2020-01-27,79.921842,80.369855,79.834190,79.834190,15013800,0.535665,-0.087651,-0.11,XLI,43,1,0,0
4,2020-01-28,80.097137,80.749685,79.727041,80.350365,14918000,1.022644,0.253227,0.32,XLI,44,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,2021-08-20,102.410004,102.940002,102.139999,102.629997,11752300,0.800003,0.219994,0.21,XLI,37671394,160857,627325,999
400,2021-08-23,103.110001,103.919998,103.059998,103.470001,8471000,0.860001,0.360001,0.35,XLI,38037577,112020,629220,703
401,2021-08-24,103.669998,104.449997,103.650002,103.989998,7593200,0.799995,0.320000,0.31,XLI,38192607,154363,630408,1184
402,2021-08-25,104.070000,105.000000,103.750000,104.660004,9404200,1.250000,0.590004,0.57,XLI,38360956,162485,631563,1138
